In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import koreanize_matplotlib
from konlpy.tag import Mecab

In [25]:
train_data = pd.read_csv("https://raw.githubusercontent.com/haram4th/data4mdai/main/hotelscom_review_train.csv")
test_data = pd.read_csv("../06machine_learning/data/hotelscom_review_test.csv")

In [26]:
train_data.head()

,description,isgood
0,사람이 너무 많고 작은 수영장과 조식 수용한계로 모두 포기하고 옆의 아이파크몰에서 ...,0
1,방도크고 깨끗하여 아주 좋았어요,1
2,매년여름휴가철마다 찾는곳이예요 너무 좋아요 점점 더 좋아지는듯 직원분들도 너무 친절...,1
3,여수에서는 제일 유명한 호텔이래요 호텔 stay 가 필요하다면 소노캄 여수도 괜찮은...,1
4,가격대비 훌륭하지만 아무래도 오래된 느낌이 많이 드네요겉이불은 세탁하니까 깨끗히나 ...,1


In [27]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53964 entries, 0 to 53963
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  53964 non-null  object
 1   isgood       53964 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 843.3+ KB


In [28]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35977 entries, 0 to 35976
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  35977 non-null  object
 1   isgood       35977 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 562.3+ KB


# 토큰화

In [29]:
docs = train_data['description']

In [30]:
mecab = Mecab()
mecab.morphs(docs[0])

['사람',
 '이',
 '너무',
 '많',
 '고',
 '작',
 '은',
 '수영장',
 '과',
 '조식',
 '수용',
 '한계',
 '로',
 '모두',
 '포기',
 '하',
 '고',
 '옆',
 '의',
 '아이파크몰',
 '에서',
 '그냥',
 '식사',
 '함']

In [31]:
type(docs)

pandas.core.series.Series

In [32]:
%%time
# 전체 문장을 토큰화 후 tokenized_docs에 저장
tokenized_docs = docs.apply(mecab.morphs)

CPU times: user 8.49 s, sys: 180 ms, total: 8.67 s
Wall time: 8.67 s


In [33]:
# import joblib
# joblib.dump(tokenized_docs, "./model/hotels_tokenized_docs")

In [34]:
# tokenized_docs = joblib.load("./model/hotels_tokenized_docs")

In [35]:
# tokenized_docs[0]

In [36]:
# 단어 인덱스 생성
token = Tokenizer(lower=False)
token.fit_on_texts(tokenized_docs)
print(len(token.word_index))

30149


In [37]:
# 문장 백터화 
X = token.texts_to_sequences(tokenized_docs)
print(X[0])

[200, 1, 30, 81, 2, 146, 10, 168, 56, 51, 3466, 3762, 47, 167, 1858, 3, 2, 231, 32, 3068, 27, 205, 221, 71]


In [38]:
y = train_data['isgood']
y

0        0
1        1
2        1
3        1
4        1
        ..
53959    1
53960    1
53961    1
53962    1
53963    1
Name: isgood, Length: 53964, dtype: int64

In [39]:
# 가장 긴 문장의 길이 구하기
max_len = max(len(i) for i in X)
print("가장 긴 문장의 길이(패딩에 사용): ", max_len)

가장 긴 문장의 길이(패딩에 사용):  738


In [40]:
# 패딩
X_padded = pad_sequences(X, maxlen=max_len, padding='post')
print(X_padded[0])

[ 200    1   30   81    2  146   10  168   56   51 3466 3762   47  167
 1858    3    2  231   32 3068   27  205  221   71    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [41]:
# 홀드아웃
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_valid, y_train, y_valid = train_test_split(X_padded, y, test_size=0.3 ,stratify=y, random_state=10)

In [43]:
# 임베딩에 입력될 단어 수
word_size = len(token.word_index) + 1 
print(word_size)

30150


# 양방향 RNN 네트워크를 이용해 텍스트 분석

In [44]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [45]:
birnn = Sequential()
birnn.add(Embedding(input_dim=word_size, output_dim=64))
birnn.add(Bidirectional(SimpleRNN(128, return_sequences=True, activation='tanh')))
birnn.add(Dropout(0.5))
birnn.add(SimpleRNN(64, activation='tanh'))
birnn.add(Dropout(0.5))
birnn.add(Dense(32, activation='relu'))
birnn.add(Dense(1, activation='sigmoid'))
# 모델 요약
birnn.build(input_shape=(None, max_len))  # 입력 형태 지정
birnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          1929600   
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 256)        49408     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, None, 256)         0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 64)                20544     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                      

In [46]:
birnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelpath = "./model/hotels_review_birnn.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=50)

In [47]:
birnn_history = birnn.fit(X_train, y_train, epochs=1000, batch_size=8,
                          validation_data=(X_valid, y_valid),
                          callbacks=[earlystop, checkpoint])

Epoch 1/1000


2024-09-19 14:04:37.618368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-09-19 14:04:38.223186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-19 14:04:38.223252: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 15603 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2024-09-19 14:04:38.224732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-19 14:04:38.224802: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

   5/4722 [..............................] - ETA: 14:39:17 - loss: 0.7670 - accuracy: 0.4250

KeyboardInterrupt: 

In [ ]:
def result_plot(x):
    # 검증셋과 학습셋의 오차를 저장합니다.
    y_vloss = x.history['val_loss']
    y_loss = x.history['loss']

    # 그래프로 표현해 봅니다.
    x_len = np.arange(len(y_loss))
    plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
    plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

    # 그래프에 그리드를 주고 레이블을 표시해 보겠습니다.
    plt.legend(loc='upper right')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

In [ ]:
result_plot(birnn_history)

# LSTM과 CNN 조합 모델로 분석

In [ ]:
from tensorflow.keras.layers import Dropout, Activation, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
# 모델의 구조를 설정합니다.
model = Sequential()
model.add(Embedding(input_dim=word_size, output_dim=64))  # 임베딩 벡터의 차원을 64로 증가
model.add(Dropout(0.5))  # Dropout 층 추가
model.add(Conv1D(128, 5, padding='valid', activation='relu', strides=1))  # Conv1D 층 추가
model.add(MaxPooling1D(pool_size=4))  # MaxPooling 층 추가
model.add(Conv1D(128, 5, padding='valid', activation='relu', strides=1))  # 추가적인 Conv1D 층
model.add(MaxPooling1D(pool_size=4))  # MaxPooling 층 추가
model.add(Bidirectional(LSTM(256, return_sequences=True, activation='tanh')))  # 양방향 LSTM 층 추가
model.add(Dropout(0.5))  # Dropout 층 추가
model.add(LSTM(128, activation='tanh'))  # 추가적인 LSTM 층
model.add(Dropout(0.5))  # Dropout 층 추가
model.add(Dense(64, activation='relu'))  # Dense 층 추가
model.add(Dense(32, activation='relu'))  # 추가적인 Dense 층
model.add(Dense(1, activation='sigmoid'))  # 출력층
# 모델 요약
model.build(input_shape=(None, max_len))  # 입력 형태 지정
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelpath = "./model/hotels_review_LSTM_CNN.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=50)

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=128,
                          validation_data=(X_valid, y_valid),
                          callbacks=[earlystop, checkpoint])

In [ ]:
# !pip install attention

In [ ]:
from attention import Attention

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input, Flatten, GlobalAveragePooling1D
from tensorflow.keras.layers import Attention
from tensorflow.keras.models import Model

# 입력 레이어
inputs = Input(shape=(max_len,))

# 임베딩 레이어
x = Embedding(input_dim=word_size, output_dim=64)(inputs)
x = Dropout(0.5)(x)

# 양방향 LSTM과 Attention
x = Bidirectional(LSTM(128, return_sequences=True))(x)
attention = Attention()([x, x])
x = Dropout(0.5)(attention)

# 추가적인 LSTM과 Attention
x = LSTM(64, return_sequences=True)(x)
attention = Attention()([x, x])
x = Dropout(0.5)(attention)

# 차원 축소
x = GlobalAveragePooling1D()(x)

# Dense 레이어
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

# 출력 레이어
outputs = Dense(1, activation='sigmoid')(x)

# 모델 정의
model = Model(inputs=inputs, outputs=outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약
model.summary()


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelpath = "./model/hotels_review_Attention.keras"
checkpoint = ModelCheckpoint(filepath=modelpath, save_best_only=True)
earlystop = EarlyStopping(patience=50)

In [ ]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=128,
                          validation_data=(X_valid, y_valid),
                          callbacks=[earlystop, checkpoint])

In [ ]:
result_plot(history)

# 머신러닝 나이브 베이즈와 비교를 위해 비교 분석

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
def tokenizer(x):
    token = mecab.morphs(x)
    return token

In [ ]:
X = train_data['description']
y = train_data['isgood']

In [ ]:
tfidf_cv = TfidfVectorizer(tokenizer=tokenizer, max_df=0.8, min_df=4, ngram_range=(1,3))
X_tfidf = tfidf_cv.fit_transform(X)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_tfidf, y, test_size=0.4, random_state=7)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=7)

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
pred = mnb.predict(X_valid)
print(classification_report(y_valid, pred))
test_pred = mnb.predict(X_test)
print("="*40, "test result", "="*40)
print(classification_report(y_test, test_pred))

# 

In [ ]:
import joblib
joblib.dump(mecab, "./model/mecab_hotels_model")
joblib.dump(mnb, "./model/Naive Bayes_hotels_model")
joblib.dump(tfidf_cv, "./model/tfidf_cv_hotels_model")